# Finetuning F&B
F&B is a Sentiment Analysis dataset with 3 possible labels: `positive`, `negative`, and `neutral`

In [1]:
import os, sys
sys.path.append('../')
os.chdir('../')

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

from utils.forward_fn import forward_sequence_classification
from utils.metrics import document_sentiment_metrics_fn
from utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

d:\coding\python\indonlu\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [3]:
# Set random seed
set_seed(26092020)

In [4]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)
print(torch.cuda.is_available())

2.5.1+cu121
4.46.3
True


# Load Model

In [5]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p2')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p2', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
count_param(model)

124443651

# Prepare Dataset

In [8]:
train_dataset_path = r"d:\coding\python\indonlu\dataset\fnb\food_quality\train_preprocess.csv"
valid_dataset_path = r"d:\coding\python\indonlu\dataset\fnb\food_quality\valid_preprocess.csv"
test_dataset_path  = r"d:\coding\python\indonlu\dataset\fnb\food_quality\test_preprocess_masked_label.csv"
test_valid_dataset_path = r"d:\coding\python\indonlu\dataset\fnb\food_quality\test_preprocess.csv"

In [9]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)
test_valid_dataset = DocumentSentimentDataset(test_valid_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=128, batch_size=8, num_workers=0, shuffle=True)  
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=128, batch_size=8, num_workers=0, shuffle=False)  
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=128, batch_size=8, num_workers=0, shuffle=False)
test_valid_loader = DocumentSentimentDataLoader(dataset=test_valid_dataset, max_seq_len=128, batch_size=8, num_workers=0, shuffle=False)

In [10]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


# Test model on sample sentences

In [11]:
text = 'Mie goreng terenak yang pernah saya makan tapi mahal dan pelayanan tidak bagus'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Mie goreng terenak yang pernah saya makan tapi mahal dan pelayanan tidak bagus | Label : positive (36.708%)


In [12]:
text = 'harganya murah banget tapi rasanya biasa aja'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: harganya murah banget tapi rasanya biasa aja | Label : positive (42.599%)


In [13]:
text = 'gak akan balik kesini lagi makanannya enak banget sampai buat sakit perut'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: gak akan balik kesini lagi makanannya enak banget sampai buat sakit perut | Label : positive (43.699%)


# Fine Tuning & Evaluation

In [14]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

2.5.1+cu121
CUDA available: True
CUDA version: 12.1


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [16]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels_train = train_dataset.labels

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1, 2]),
    y=np.array(labels_train)
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
model = model.to(device)

In [17]:
print("Model embedding device:",
      model.bert.embeddings.word_embeddings.weight.device)

Model embedding device: cuda:0


In [18]:
import os
import torch
from tqdm import tqdm

# =========================
# KONFIGURASI
# =========================
n_epochs = 10
save_dir = "model_foood"   # ganti sesuai aspek
os.makedirs(save_dir, exist_ok=True)

best_valid_loss = float("inf")

# =========================
# TRAINING LOOP
# =========================
for epoch in range(n_epochs):

    # =========================
    # TRAINING
    # =========================
    model.train()
    total_train_loss = 0
    train_preds, train_labels = [], []

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} TRAIN")

    for batch_data in train_pbar:
        optimizer.zero_grad()

        loss, batch_pred, batch_label = forward_sequence_classification(
            model=model,
            batch_data=batch_data,
            device=device,
            criterion=criterion
        )

        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_preds.extend(batch_pred)
        train_labels.extend(batch_label)

        train_pbar.set_postfix({
            "loss": f"{total_train_loss / (len(train_preds)//len(batch_pred)):.4f}",
            "lr": f"{get_lr(optimizer):.2e}"
        })

    train_metrics = document_sentiment_metrics_fn(train_preds, train_labels)

    print(
        f"(Epoch {epoch+1}) "
        f"TRAIN LOSS: {total_train_loss / len(train_loader):.4f} "
        f"{metrics_to_string(train_metrics)} "
        f"LR:{get_lr(optimizer):.2e}"
    )

    # =========================
    # VALIDATION
    # =========================
    model.eval()
    total_valid_loss = 0
    valid_preds, valid_labels = [], []

    with torch.no_grad():
        valid_pbar = tqdm(valid_loader, desc=f"Epoch {epoch+1} VALID")

        for batch_data in valid_pbar:
            loss, batch_pred, batch_label = forward_sequence_classification(
                model=model,
                batch_data=batch_data,
                device=device,
            criterion=criterion
            )

            total_valid_loss += loss.item()
            valid_preds.extend(batch_pred)
            valid_labels.extend(batch_label)

            valid_pbar.set_postfix({
                "loss": f"{total_valid_loss / (len(valid_preds)//len(batch_pred)):.4f}"
            })

    avg_valid_loss = total_valid_loss / len(valid_loader)
    valid_metrics = document_sentiment_metrics_fn(valid_preds, valid_labels)

    print(
        f"(Epoch {epoch+1}) "
        f"VALID LOSS: {avg_valid_loss:.4f} "
        f"{metrics_to_string(valid_metrics)}"
    )

    # =========================
    # SIMPAN MODEL TERBAIK
    # =========================
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss

        print(
            f"✔ Model terbaik disimpan "
            f"(Epoch {epoch+1}, VALID LOSS={best_valid_loss:.4f})"
        )

        model.save_pretrained(save_dir)
        tokenizer.save_pretrained(save_dir)

print("\nTraining selesai.")
print(f"Model terbaik tersimpan di folder: {save_dir}")

Epoch 1 TRAIN: 100%|██████████| 273/273 [01:03<00:00,  4.27it/s, loss=0.5556, lr=2.00e-05]


(Epoch 1) TRAIN LOSS: 0.5556 ACC:0.77 F1:0.77 REC:0.77 PRE:0.77 LR:2.00e-05


Epoch 1 VALID: 100%|██████████| 59/59 [00:03<00:00, 19.44it/s, loss=0.1905]


(Epoch 1) VALID LOSS: 0.3778 ACC:0.85 F1:0.85 REC:0.85 PRE:0.85
✔ Model terbaik disimpan (Epoch 1, VALID LOSS=0.3778)


Epoch 2 TRAIN: 100%|██████████| 273/273 [01:02<00:00,  4.36it/s, loss=0.2899, lr=2.00e-05]


...

(Epoch 9) VALID LOSS: 0.6538 ACC:0.84 F1:0.84 REC:0.84 PRE:0.84


Epoch 10 TRAIN: 100%|██████████| 273/273 [01:03<00:00,  4.31it/s, loss=0.0151, lr=2.00e-05]


(Epoch 10) TRAIN LOSS: 0.0151 ACC:1.00 F1:1.00 REC:1.00 PRE:1.00 LR:2.00e-05


Epoch 10 VALID: 100%|██████████| 59/59 [00:02<00:00, 19.89it/s, loss=0.4321]

(Epoch 10) VALID LOSS: 0.8568 ACC:0.84 F1:0.84 REC:0.84 PRE:0.85

Training selesai.
Model terbaik tersimpan di folder: model_foood
